### Useful

In [1]:
%%bash 
uname -a 
date
pwd

Darwin Air-de-j.lan 20.2.0 Darwin Kernel Version 20.2.0: Wed Dec  2 20:39:59 PST 2020; root:xnu-7195.60.75~1/RELEASE_X86_64 x86_64
Fri May  6 09:21:51 CEST 2022
/Users/poderozita/z2022_1/REPOSITORY/NVCC


## Challenge 

In [2]:
%%writefile smallest.cu 
#include <stdio.h>

__global__
void cuda_hello()
{
  printf(">> [smallest] Hello from GPU!\n");
}

int main()
{
    printf(">> [smallest] Hello from CPU!\n");   
    cuda_hello<<<1,1>>>();
    return 0;
}

Overwriting smallest.cu


In [3]:
%%writefile CMakeLists.txt 

cmake_minimum_required(VERSION 3.8 FATAL_ERROR)
project(hello LANGUAGES CXX CUDA)
enable_language(CUDA)
add_executable(smallest.x ../smallest.cu)

## CMAKE_CUDA_ARCHITECTURES now detected for NVCC, empty CUDA_ARCHITECTURES not allowed.
set_property(TARGET smallest.x PROPERTY CUDA_ARCHITECTURES OFF) 

enable_testing()
add_test(NAME smallest COMMAND smallest.x) 

Overwriting CMakeLists.txt


## A solution 

In [4]:
%%writefile GET_NVCC.sh 

## https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html
SETUP_NVCC()
{ 
  echo "wget..."
  wget https://developer.download.nvidia.com/compute/cuda/redist/cuda_nvcc/linux-x86_64/cuda_nvcc-linux-x86_64-${RELEASE}-archive.tar.xz
  
  echo "tar..."
  rm -rf cuda_nvcc-linux-x86_64-${RELEASE}-archive
  tar -xf cuda_nvcc-linux-x86_64-${RELEASE}-archive.tar.xz 
}


SETUP_CUDART()
{ 
  echo "wget..."
  wget https://developer.download.nvidia.com/compute/cuda/redist/cuda_cudart/linux-x86_64/cuda_cudart-linux-x86_64-${RELEASE}-archive.tar.xz
  
  echo "tar..."
  rm -rf cuda_cudart-linux-x86_64-${RELEASE}-archive
  tar -xf cuda_cudart-linux-x86_64-${RELEASE}-archive.tar.xz
}


INIT()
{
  ## 1. SETUP 
  g++ --version   ## g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
  cmake --version ## cmake version 3.16.3

  NVCC_PATH=${PWD}/cuda_nvcc-linux-x86_64-${RELEASE}-archive
  CUDART_PATH=${PWD}/cuda_cudart-linux-x86_64-${RELEASE}-archive

  NVCC=$NVCC_PATH/bin/nvcc
  $NVCC --version

  rm -rf BUILD
  mkdir BUILD 
  cd BUILD 

  ## 2. COMPILATION
  ## 2.A. SIMPLEST  
  $NVCC -o smallest.x ../smallest.cu -I$CUDART_PATH/include -L$CUDART_PATH/lib
  ./smallest.x 
  rm smallest.x  

  ## 2.B. EASIEST  
  export CPLUS_INCLUDE_PATH=$CPLUS_INCLUDE_PATH:$CUDART_PATH/include 
  export LIBRARY_PATH=$LIBRARY_PATH:$CUDART_PATH/lib 
  export CUDACXX=$NVCC 

  $CUDACXX  -o smallest.x ../smallest.cu  
  ./smallest.x 
  rm smallest.x
    
  ## 2.C. FANCIEST (https://www.collinsdictionary.com/dictionary/english/fanciest)
  rm -rf CMakeCache.txt CMakeFiles  
  cmake ..
  make
  ctest
}


RELEASE=11.5.50 
SETUP_NVCC
SETUP_CUDART
INIT


Overwriting GET_NVCC.sh


In [5]:
!bash GET_NVCC.sh # To be runned on 'mybinder' 

## Containerization

In [6]:
%%writefile Dockerfile.ci 

FROM ubuntu:focal 
RUN apt-get --yes -qq update 
RUN DEBIAN_FRONTEND=noninteractive TZ=Etc/UTC apt-get -y install tzdata
RUN apt-get --yes -qq update 

RUN apt-get --yes -qq install wget xz-utils
RUN apt-get --yes -qq install build-essential  
RUN apt-get --yes -qq install gcc g++  
RUN apt-get --yes -qq install cmake 
RUN apt-get --yes -qq install cmake-curses-gui ## ccmake 
RUN apt-get --yes -qq clean
RUN rm -rf /var/lib/apt/lists/*

Overwriting Dockerfile.ci


In [7]:
%%writefile workflow.sh 


DOCKER_BUILD()
{
  docker build . \
    --file Dockerfile.ci \
    --tag $IMAGE_NAME  \
    > /dev/null 
}


DOCKER_IMAGES()
{
  IMAGE_ID=$(docker images --filter=reference=$IMAGE_NAME --format "{{.ID}}") 
  echo "IMAGE_ID:", $IMAGE_ID    
}


DOCKER_RUN()
{
  docker run \
    --rm \
    --name ${CONTAINER_NAME} \
    --volume ${PWD}:/home \
    --workdir /home \
    -i ${IMAGE_NAME} \
    ${TOBEEXECUTED}
}


CONTAINER_NAME=nvcc_container
IMAGE_NAME=nvcc_image 

DOCKER_BUILD 

TOBEEXECUTED="ls -la"
DOCKER_RUN

TOBEEXECUTED="pwd"
DOCKER_RUN

TOBEEXECUTED="bash GET_NVCC.sh"
DOCKER_RUN

#
# docker run --rm --name nvcc_container --volume ${PWD}:/home --workdir /home -i nvcc_image bash GET_NVCC.sh
# 

Overwriting workflow.sh


In [8]:
#!bash workflow.sh # To be runned 'localy' (SEE '.github/workflows/dockerci.yml')  

 => [internal] load build definition from Dockerfile.ci                    0.0s
 => => transferring dockerfile: 487B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/ubuntu:focal            1.5s
 => [auth] library/ubuntu:pull token for registry-1.docker.io              0.0s
 => [ 1/11] FROM docker.io/library/ubuntu:focal@sha256:47f14534bda344d9fe  0.0s
 => CACHED [ 2/11] RUN apt-get --yes -qq update                            0.0s
 => CACHED [ 3/11] RUN DEBIAN_FRONTEND=noninteractive TZ=Etc/UTC apt-get   0.0s
 => CACHED [ 4/11] RUN apt-get --yes -qq update                            0.0s
 => CACHED [ 5/11] RUN apt-get --yes -qq install wget xz-utils             0.0s
 => CACHED [ 6/11] RUN apt-get --yes -qq install build-essential           0.0s
 => CACHED [ 7/11] RUN apt-get --yes -qq


Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them
total 147896
drwxr-xr-x  9 root  root      288 May  6 06:23 BUILD
-rw-r--r--  1 root  root      364 May  6 07:21 CMakeLists.txt
-rw-r--r--  1 root  root      440 May  6 07:21 Dockerfile.ci
-rw-r--r--  1 root  root     1637 May  6 07:21 GET_NVCC.sh
-rw-r--r--  1 root  root     1456 May  5 08:33 cmake_install.cmake
drwxr-xr-x  6 25503 tape      192 Sep 14  2021 cuda_cudart-linux-x86_64-11.5.50-archive
-rw-r--r--  1 root  root   835692 Oct 16  2021 cuda_cudart-linux-x86_64-11.5.50-archive.tar.xz
drwxr-xr-x  7 25503 tape      224 Sep 14  2021 cuda_nvcc-linux-x86_64-11.5.50-archive
-rw-r--r--  1 root  root 36310316 Oct 16  2021 cuda_nvcc-linux-x86_64-11.5.50-archive.tar.xz
-rw-r--r--  1 root  root     8200 May  6 07:21 nvcc_ubuntufocal.ipynb
-rw-r--r--  1 root  root      205 May  6 07:21 smallest.cu
-rw-r--r--  1 root  root      707 May  6 07:21 workflow.sh
/home
wget...
--2022-05-06 07:21:

 30250K .......... .......... .......... .......... .......... 85% 1.34M 5s
tar...
2022-05-06 07:22:33 (1021 KB/s) - 'cuda_nvcc-linux-x86_64-11.5.50-archive.tar.xz.4' saved [36310316/36310316]

wget...
--2022-05-06 07:23:23--  https://developer.download.nvidia.com/compute/cuda/redist/cuda_cudart/linux-x86_64/cuda_cudart-linux-x86_64-11.5.50-archive.tar.xz


Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 835692 (816K) [application/octet-stream]
Saving to: 'cuda_cudart-linux-x86_64-11.5.50-archive.tar.xz.5'
... 
2022-05-06 07:23:24 (854 KB/s) - 'cuda_cudart-linux-x86_64-11.5.50-archive.tar.xz.5' saved [835692/835692]

tar...
g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

cmake version 3.16.3

CMake suite maintained and supported by Kitware (kitware.com/cmake).
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Mon_Sep_13_19:13:29_PDT_2021
Cuda compilation tools, release 11.5, V11.5.50
Build cuda_11.5.r11.

## Done!!